# read data

In [20]:
import pandas as pd
import numpy as np
ratings = pd.read_csv('u.data', header=None, sep='\t', names=['uid', 'iid', 'val', 'tag'])
ratings.head()

,uid,iid,val,tag
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [21]:
users = pd.read_csv('u.user', header=None, sep='|', names=['uid', 'age', 'gender', 'occup', 'zip'])
users.head()
gender_dict = {}
age_dict = {}
occup_dict = {}
for i in range(len(users.uid)):
    gender_dict[users.uid[i] - 1] = users.gender[i]
    occup_dict[users.uid[i] - 1] = users.occup[i]
    age_dict[users.uid[i] - 1] = users.age[i]

print(age_dict)
print(occup_dict)
print(gender_dict)
# np.save('user_gender', gender_dict)


{0: 24, 1: 53, 2: 23, 3: 24, 4: 33, 5: 42, 6: 57, 7: 36, 8: 29, 9: 53, 10: 39, 11: 28, 12: 47, 13: 45, 14: 49, 15: 21, 16: 30, 17: 35, 18: 40, 19: 42, 20: 26, 21: 25, 22: 30, 23: 21, 24: 39, 25: 49, 26: 40, 27: 32, 28: 41, 29: 7, 30: 24, 31: 28, 32: 23, 33: 38, 34: 20, 35: 19, 36: 23, 37: 28, 38: 41, 39: 38, 40: 33, 41: 30, 42: 29, 43: 26, 44: 29, 45: 27, 46: 53, 47: 45, 48: 23, 49: 21, 50: 28, 51: 18, 52: 26, 53: 22, 54: 37, 55: 25, 56: 16, 57: 27, 58: 49, 59: 50, 60: 36, 61: 27, 62: 31, 63: 32, 64: 51, 65: 23, 66: 17, 67: 19, 68: 24, 69: 27, 70: 39, 71: 48, 72: 24, 73: 39, 74: 24, 75: 20, 76: 30, 77: 26, 78: 39, 79: 34, 80: 21, 81: 50, 82: 40, 83: 32, 84: 51, 85: 26, 86: 47, 87: 49, 88: 43, 89: 60, 90: 55, 91: 32, 92: 48, 93: 26, 94: 31, 95: 25, 96: 43, 97: 49, 98: 20, 99: 36, 100: 15, 101: 38, 102: 26, 103: 27, 104: 24, 105: 61, 106: 39, 107: 44, 108: 29, 109: 19, 110: 57, 111: 30, 112: 47, 113: 27, 114: 31, 115: 40, 116: 20, 117: 21, 118: 32, 119: 47, 120: 54, 121: 32, 122: 48, 123

# data info

In [22]:
print('Number of ratings:', len(ratings))
print('Number of users:', len(ratings.uid.unique()))
print('Number of movies:', len(ratings.iid.unique()))

print(max(ratings.uid.unique()))

Number of ratings: 100000
Number of users: 943
Number of movies: 1682
943


In [23]:
print('male:', len(users[users.gender == 'M']))
print('female:', len(users[users.gender == 'F']))

male: 670
female: 273


In [24]:
print(ratings.groupby('uid').agg('count'))

     iid  val  tag
uid               
1    272  272  272
2     62   62   62
3     54   54   54
4     24   24   24
5    175  175  175
..   ...  ...  ...
939   49   49   49
940  107  107  107
941   22   22   22
942   79   79   79
943  168  168  168

[943 rows x 3 columns]


# filter interaction < 5

In [25]:
print('user:', min(ratings.groupby('uid').agg('count').iid))
print('item:', min(ratings.groupby('iid').agg('count').uid))

user: 20
item: 1


In [26]:
from tqdm import tqdm

low = 5
while True:
    n_filter = 0
    # drop item
    print('>>> item:')
    user_data = ratings.groupby('iid').agg(list)
    for i in tqdm(ratings.iid.unique()):
        if len(user_data['uid'][i]) < low:
            ratings = ratings[ratings.iid.values != i]
            n_filter += 1

    # drop user
    print('>>> user:')
    item_data = ratings.groupby('uid').agg(list)
    for i in tqdm(ratings.uid.unique()):
        if len(item_data['iid'][i]) < low:
            ratings = ratings[ratings.uid.values != i]
            n_filter += 1
    
    # terminate
    if n_filter == 0:
        break

# reset index
ratings.reset_index(drop=True, inplace=True)

>>> item:


100%|██████████| 1682/1682 [00:00<00:00, 4967.83it/s] 


>>> user:


100%|██████████| 943/943 [00:00<00:00, 235178.30it/s]


>>> item:


100%|██████████| 1349/1349 [00:00<00:00, 228297.13it/s]


>>> user:


100%|██████████| 943/943 [00:00<00:00, 246585.33it/s]


In [27]:
print('user:', min(ratings.groupby('uid').agg('count').iid))
print('item:', min(ratings.groupby('iid').agg('count').uid))

user: 19
item: 5


# squeeze id

In [28]:
print(f'user id: {max(ratings.uid.unique())}/{len(ratings.uid.unique())}')
print(f'item id: {max(ratings.iid.unique())}/{len(ratings.iid.unique())}')

user id: 943/943
item id: 1620/1349


In [29]:
print(ratings)

       uid   iid  val        tag
0      196   242    3  881250949
1      186   302    3  891717742
2       22   377    1  878887116
3      244    51    2  880606923
4      166   346    1  886397596
...    ...   ...  ...        ...
99282  880   476    3  880175444
99283  716   204    5  879795543
99284  276  1090    1  874795795
99285   13   225    2  882399156
99286   12   203    3  879959583

[99287 rows x 4 columns]


In [30]:
import numpy as np

# squeeze uid
ratings.uid -= 1

# squeeze iid
item_dict = {}
for i, ori in enumerate(ratings['iid'].unique()):
    item_dict[ori] = i
    
np.save('item_dict', item_dict)

ratings.iid = ratings.iid.map(item_dict)

# save file
ratings.sort_values(by=['uid', 'iid']).to_csv('ratings.csv', index=False)

In [32]:
import pandas as pd
new_pdr = pd.read_csv("ratings.csv")
new_pdr = new_pdr[['uid','iid','val']]

new_pdr['val'] = 1
print(new_pdr)

       uid   iid  val
0        0     0    1
1        0     3    1
2        0     6    1
3        0     9    1
4        0    10    1
...    ...   ...  ...
99282  942  1025    1
99283  942  1058    1
99284  942  1061    1
99285  942  1077    1
99286  942  1179    1

[99287 rows x 3 columns]


In [52]:
from tqdm import tqdm
import time 
import pandas as pd
import random

train_ratio = 0.8
train_df = pd.DataFrame([], columns=['uid', 'iid', 'val'])
test_df = pd.DataFrame([], columns=['uid', 'iid', 'val'])

train_index = []
test_index = []


for i in range(len(new_pdr.uid.unique())):
    t1 = time.time()
    indices = new_pdr[new_pdr.uid == i].index
    index_list = indices.to_list()
    
    total = len(indices)

    n_train = int(total * train_ratio)
    
    train_idx = random.sample(index_list, n_train)
    test_idx = list(set(index_list) - set(train_idx))
    
    train_index.extend(train_idx)
    test_index.extend(test_idx)
    
    # if i%1000 == 0:
    #     print(i)

train_index = np.array(train_index)
test_index = np.array(test_index)

user_train = new_pdr.iloc[np.sort(train_index), :]
user_test = new_pdr.iloc[np.sort(test_index), :]

train_df = pd.concat([train_df,user_train], ignore_index=True)
test_df = pd.concat([test_df, user_test], ignore_index=True)  

print(train_df)
print(test_df)

# c=9/0
# train_df['val'] = np.float16(train_df['rating'])
# test_df['val'] = np.float16(test_df['rating'])

train_df[['uid', 'iid', 'val']].to_csv('train.csv', index=False)
test_df[['uid', 'iid', 'val']].to_csv('test.csv', index=False)

       uid   iid val
0        0     0   1
1        0     6   1
2        0     9   1
3        0    10   1
4        0    12   1
...    ...   ...  ..
79040  942  1025   1
79041  942  1058   1
79042  942  1061   1
79043  942  1077   1
79044  942  1179   1

[79045 rows x 3 columns]
       uid   iid val
0        0     3   1
1        0    30   1
2        0    59   1
3        0    61   1
4        0    68   1
...    ...   ...  ..
20237  942   622   1
20238  942   656   1
20239  942   870   1
20240  942   987   1
20241  942  1011   1

[20242 rows x 3 columns]


# divide train:test

In [50]:
print(f'user id: {max(ratings.uid.unique())}/{len(ratings.uid.unique())}')
print(f'item id: {max(ratings.iid.unique())}/{len(ratings.iid.unique())}')

user id: 942/943
item id: 1348/1349


In [21]:
import pandas as pd
ll = pd.read_csv("ratings.csv")
print(ll)

       uid   iid  val        tag
0        0     0    5  889751633
1        0     3    4  878543275
2        0     6    4  878542441
3        0     9    5  878543541
4        0    10    4  874965954
...    ...   ...  ...        ...
99282  942  1025    4  888640250
99283  942  1058    3  888640250
99284  942  1061    1  888640027
99285  942  1077    3  888640275
99286  942  1179    4  888639954

[99287 rows x 4 columns]


In [54]:
import pandas as pd
train_data = pd.read_csv("train.csv")
print(train_data)

       uid   iid  val
0        0     0    1
1        0     6    1
2        0     9    1
3        0    10    1
4        0    12    1
...    ...   ...  ...
79040  942  1025    1
79041  942  1058    1
79042  942  1061    1
79043  942  1077    1
79044  942  1179    1

[79045 rows x 3 columns]


In [12]:
print(train_data.iloc[:, 0])

0          0
1          0
2          0
3          0
4          0
        ... 
79039    942
79040    942
79041    942
79042    942
79043    942
Name: 0, Length: 79044, dtype: int64


In [55]:
user_num = train_data['uid'].max() + 1
item_num = train_data['iid'].max() + 1

# train_data = train_data.values.tolist()
train_data = train_data.values.tolist()

pos_dict = {}

for x in train_data:
    if x[0] not in pos_dict.keys():
        pos_dict[x[0]] = []
    pos_dict[x[0]].append(x[1])

np.save('pos_dict.npy', pos_dict)

In [56]:
d = np.load('pos_dict.npy', allow_pickle=True).item()
print(d[0])

[0, 6, 9, 10, 12, 13, 14, 22, 23, 24, 26, 29, 31, 32, 33, 34, 38, 39, 44, 46, 47, 49, 52, 53, 56, 64, 65, 71, 73, 75, 76, 77, 79, 83, 86, 88, 93, 94, 100, 106, 112, 113, 116, 117, 118, 122, 130, 139, 146, 147, 148, 156, 157, 166, 169, 170, 176, 177, 181, 182, 188, 191, 195, 199, 203, 208, 209, 215, 216, 217, 218, 220, 221, 224, 229, 233, 235, 239, 247, 250, 253, 254, 256, 260, 263, 269, 275, 277, 288, 292, 296, 297, 303, 305, 307, 311, 313, 317, 320, 321, 324, 328, 329, 330, 337, 338, 347, 351, 356, 357, 359, 363, 367, 371, 373, 379, 382, 385, 387, 390, 398, 402, 404, 407, 422, 431, 442, 443, 446, 450, 460, 465, 467, 469, 478, 489, 490, 497, 499, 500, 501, 504, 519, 520, 524, 525, 527, 532, 538, 541, 543, 544, 546, 549, 552, 553, 554, 563, 568, 574, 586, 588, 592, 595, 608, 609, 611, 619, 621, 642, 655, 658, 661, 681, 697, 712, 715, 716, 717, 725, 731, 748, 760, 765, 766, 768, 785, 793, 805, 808, 843, 848, 850, 851, 856, 858, 892, 910, 918, 922, 923, 926, 934, 963, 980, 984, 987, 1030,

In [4]:
import torch

t = torch.tensor([3,3,1])
print(t.shape)
print(torch.log(t).sum())

torch.Size([3])
tensor(2.1972)


In [12]:
import numpy as np
def hit(gt_items, pred_items):
    hr = 0
    for gt_item in gt_items:
        if gt_item in pred_items:
            hr = hr + 1

    return hr/len(gt_items)

def ndcg(gt_items, pred_items):
    dcg = 0
    idcg = 0
    
    for gt_item in gt_items:
        if gt_item in pred_items:
            index = pred_items.index(gt_item)
            dcg = dcg + np.reciprocal(np.log2(index+2))
            
    for index in range(len(gt_items)):
        idcg = idcg + np.reciprocal(np.log2(index+2))
    
    print(dcg)
    print(idcg)
            
    return dcg/idcg

In [17]:
gt_items = [1,2,3,4,5,6,7,8,9,10]
pred_items = [11,11,11,11,11,11,11,9,10]

print(hit(gt_items, pred_items))
print(ndcg(gt_items, pred_items))

0.2
0.6164948724497099
4.543559338088346
0.13568544539116628
